In [ ]:
# ANGELO DAVID PADILLA ROMERO_BOOTCAMP-IA_NIVEL-AVANZADO_TALENTOTECH - INSTRUCTOR: CRISTIAN CAMILO TIRADO CIFUENTES - 2024

!pip install scikit-surprise pandas numpy matplotlib kaggle requests
import pandas as pd
from surprise import Dataset, Reader, KNNBasic, SVD
from surprise.model_selection import train_test_split, GridSearchCV
from surprise import accuracy
import subprocess

In [ ]:
# Configuración de Kaggle
kaggle_url = "arashnic/book-recommendation-dataset"
file_name = "Ratings.csv"

In [ ]:
# Descargar y descomprimir el dataset desde Kaggle
subprocess.run(['kaggle', 'datasets', 'download', '-d', kaggle_url, '-f', file_name])
subprocess.run(['unzip', f'{file_name}.zip'])

CompletedProcess(args=['unzip', 'Ratings.csv.zip'], returncode=1)

In [ ]:
# Cargar el dataset
data = pd.read_csv(file_name)
print("Primeras filas del dataset:")
print(data.head())
print("Forma del dataset:", data.shape)

Primeras filas del dataset:
   User-ID        ISBN  Book-Rating
0   276725  034545104X            0
1   276726  0155061224            5
2   276727  0446520802            0
3   276729  052165615X            3
4   276729  0521795028            6
Forma del dataset: (1149780, 3)


In [ ]:
# Filtrar usuarios con menos de 20 ratings
ratings_user = data['User-ID'].value_counts()
filtered_data = data[~data['User-ID'].isin(ratings_user[ratings_user < 20].index)]
print("Forma del dataset tras filtrar usuarios:", filtered_data.shape)

# Filtrar libros con menos de 50 ratings
ratings_book = filtered_data['ISBN'].value_counts()
filtered_data = filtered_data[~filtered_data['ISBN'].isin(ratings_book[ratings_book < 50].index)]
print("Forma del dataset tras filtrar libros:", filtered_data.shape)

Forma del dataset tras filtrar usuarios: (888884, 3)
Forma del dataset tras filtrar libros: (140483, 3)


In [ ]:
# Comprobar el intervalo de ratings
print("Distribución de ratings:")
print(filtered_data["Book-Rating"].value_counts(True))

Distribución de ratings:
Book-Rating
0     0.669732
8     0.081718
10    0.068684
9     0.061580
7     0.053152
5     0.029669
6     0.023718
4     0.005268
3     0.003488
2     0.001808
1     0.001182
Name: proportion, dtype: float64


In [ ]:
# Convertir datos al formato de Surprise
reader = Reader(rating_scale=(0, 10))
data_surprise = Dataset.load_from_df(filtered_data[['User-ID', 'ISBN', 'Book-Rating']], reader)

In [ ]:
# Dividir los datos en conjunto de entrenamiento y prueba
trainset, testset = train_test_split(data_surprise, test_size=0.2)

In [ ]:
# Búsqueda de hiperparámetros con Grid Search para KNN
param_grid = {'k': [20, 30, 40], 'min_k': [1, 5, 10]}
gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=3, refit=True, n_jobs=-1)
gs.fit(data_surprise)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
# Mejor configuración del modelo KNN
print("Mejores hiperparámetros para KNN:", gs.best_params['rmse'])
print("Scoring de la mejor configuración de KNN:", gs.best_score)

Mejores hiperparámetros para KNN: {'k': 40, 'min_k': 10}
Scoring de la mejor configuración de KNN: {'rmse': 3.775979540104649}


In [ ]:
# Generar predicciones para el dataset
predictions = gs.test(testset)
predictions_df = pd.DataFrame(predictions)
predictions_df["est"] = predictions_df["est"].round(1)
predictions_df.rename(columns={
    "uid": "User-ID",
    "iid": "ISBN",
    "r_ui": "Expected-Book-Rating",
    "est": "Predicted-Book-Rating"
}, inplace=True)

In [ ]:
# Ejemplo de predicción para un usuario y un libro
user_id = 157655
book_id = "0446603716"
prediction = gs.predict(user_id, book_id)
print(f"Calificación predicha para el usuario '{user_id}' sobre el libro '{book_id}': {round(prediction.est, 1)}")

Calificación predicha para el usuario '157655' sobre el libro '0446603716': 7.1


In [ ]:
# Evaluación del modelo SVD
model_svd = SVD()
trainset, testset = train_test_split(data_surprise, test_size=0.2)
model_svd.fit(trainset)
predictions_svd = model_svd.test(testset)
print("RMSE del modelo SVD:", accuracy.rmse(predictions_svd))

RMSE: 3.6350
RMSE del modelo SVD: 3.6349769957079334
